```
rawaudio -> mfcc -> encoder -> articulator -> mfcc -> mseloss
             |       |
             `-------`-------> discriminator
``` 

In [1]:
import train
import importlib
importlib.reload(train)

from train import PinkTromboneModel
import torch
import torchaudio
from torch import nn

model = PinkTromboneModel(batch_size=4)

In [2]:
model = PinkTromboneModel.load_from_checkpoint('lightning_logs/version_8/checkpoints/last.ckpt')

In [10]:
model.setup("train")
dataloader = model.train_dataloader()
batch = next(iter(dataloader))

In [11]:
import IPython.display as ipd
ipd.Audio(data=batch[0], rate=22500)

In [189]:
from tqdm import tqdm
model.setup("train")
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
train_loader = model.train_dataloader()
pbar = tqdm(train_loader)
for batch in pbar:
    x = model.compute_loss(batch)
    loss = x['rec_loss'] + x['est_loss']
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    pbar.set_postfix(
        rec_loss=x['rec_loss'].detach().cpu().item(),
        est_loss=x['est_loss'].detach().cpu().item(), end='')


  0%|          | 22/7135 [01:43<9:18:11,  4.71s/it, end=, est_loss=205, rec_loss=33.2] 


KeyboardInterrupt: 

In [13]:
targets = model.transform(batch)
x = model.encoder(targets)
estimates = model.decoder(x)
x = model.articulator(x).to(x.dtype).to(x.device)
x = torchaudio.functional.resample(x, 44100, 22050)
x = x / torch.max(torch.abs(x), axis=1, keepdim=True).values
audio = x
outputs = model.transform(x)
length = min(outputs.shape[2], targets.shape[2])
rec_loss = model.criterion(outputs[:, :, :length], targets[:, :, :length])
est_loss = model.criterion(outputs[:, :, :length], estimates[:, :, :length])


/home/ubuntu/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3, 1, 124])) that is different to the input size (torch.Size([3, 32, 124])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [19]:
import IPython.display as ipd
ipd.Audio(data=audio[0], rate=44100)

In [62]:
loss = est_loss + rec_loss
loss

tensor(3521.7788, grad_fn=<AddBackward0>)

In [72]:
import IPython.display as ipd
ipd.Audio(data=outputs[9], rate=44100)

In [21]:
import IPython.display as ipd
ipd.Audio(data=batch[0], rate=22050)

In [50]:
batch.shape[1] / 256

126.0546875

In [47]:
x.shape

torch.Size([32, 32, 127])

In [10]:
import IPython.display as ipd
ipd.Audio(data=z, rate=16000)